In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install win10toast

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install openpyxl

#### pip install plyer

In [21]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime, timedelta
import pandas as pd
from openpyxl import load_workbook

class TaskManager:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Task Manager")
        # Initialize tasks
        self.tasks, self.completed_tasks, self.expired_tasks = self.load_tasks_from_excel("python.xlsx")

        # Create a notebook (tabbed interface)
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill='both', expand=True)

        # Add two frames/pages to the notebook
        self.add_task_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.add_task_frame, text='Add / Update / Delete Task')
        self.create_add_task_widgets()
        self.create_view_task_widgets()
        self.root.mainloop()

    def create_add_task_widgets(self):
        # Labels
        labels = ["ID to Update/Delete:", "New ID:", "Title:", "Description:", "Deadline (dd-mm-yyyy):"]
        for i, label_text in enumerate(labels):
            label = tk.Label(self.add_task_frame, text=label_text)
            label.grid(row=i, column=0, padx=10, pady=5, sticky="e")
        
        # Entries
        self.id_to_update_delete_entry = tk.Entry(self.add_task_frame)
        self.new_id_entry = tk.Entry(self.add_task_frame)
        self.title_entry = tk.Entry(self.add_task_frame)
        self.description_entry = tk.Entry(self.add_task_frame)
        self.deadline_entry = tk.Entry(self.add_task_frame)
    
        entries = [self.id_to_update_delete_entry, self.new_id_entry, self.title_entry, self.description_entry, self.deadline_entry]
        for i, entry in enumerate(entries):
            entry.grid(row=i, column=1, padx=10, pady=5, sticky="w")
    
        # Buttons
        button_frame = ttk.Frame(self.add_task_frame)
        button_frame.grid(row=len(labels), columnspan=2, padx=10, pady=10, sticky="e")
    
        update_button = ttk.Button(button_frame, text="Update Task", command=self.update_task)
        delete_button = ttk.Button(button_frame, text="Delete Task", command=self.delete_task)
        add_button = ttk.Button(button_frame, text="Add Task", command=self.add_task)
    
        update_button.pack(side="left", padx=5)
        delete_button.pack(side="left", padx=5)
        add_button.pack(side="left", padx=5)
    
        # Set uniform padding for all widgets
        for child in self.add_task_frame.winfo_children():
            child.grid_configure(padx=5, pady=5,sticky="nsew")
            
    def create_view_task_widgets(self):
    # Define styles for the treeview
        style = ttk.Style()
        style.configure("Treeview.Heading", font=("Helvetica", 10, "bold"))
        style.configure("Treeview", font=("Helvetica", 10), rowheight=25)
        style.map("Treeview", background=[('selected', '#0078d7')])
    
        # Iterate through the task sheets and create tabs dynamically
        for sheet_name in ["Original task", "Completed task", "Expired task"]:
            frame = ttk.Frame(self.notebook)
            self.notebook.add(frame, text=sheet_name)
    
            # Create a treeview for each tab
            tasks_table = ttk.Treeview(frame, columns=("ID", "Title", "Description", "Deadline", "Remaining Time"))
            tasks_table.heading("#0", text="", anchor=tk.CENTER)
            tasks_table.heading("ID", text="ID", anchor=tk.CENTER)
            tasks_table.heading("Title", text="Title", anchor=tk.CENTER)
            tasks_table.heading("Description", text="Description", anchor=tk.CENTER)
            tasks_table.heading("Deadline", text="Deadline", anchor=tk.CENTER)
            tasks_table.heading("Remaining Time", text="Remaining Time", anchor=tk.CENTER)
    
            # Set column widths
            tasks_table.column("#0", width=1, stretch=tk.NO)  # Hide the first column
            tasks_table.column("ID", width=100, anchor=tk.CENTER)
            tasks_table.column("Title", width=200, anchor=tk.CENTER)
            tasks_table.column("Description", width=300, anchor=tk.CENTER)
            tasks_table.column("Deadline", width=150, anchor=tk.CENTER)
            tasks_table.column("Remaining Time", width=150, anchor=tk.CENTER)
    
            # Alternate row colors
            tasks_table.tag_configure("oddrow", background="#f0f0f0")
            tasks_table.tag_configure("evenrow", background="#ffffff")
            # Load tasks from the respective sheet into the table
            self.load_tasks_to_table(sheet_name, tasks_table)
            tasks_table.pack(fill='both', expand=True)


    def delete_task(self):
        id_to_delete = self.id_to_update_delete_entry.get()
        if id_to_delete:
            task_index = self.search_task_id(id_to_delete)
            if task_index is not None:
                response = messagebox.askyesno("Task Completion", "Is the task completed?")  
                if response:
                    if self.completed_tasks:
                        self.completed_tasks.append(self.tasks.pop(task_index))
                    else:
                        self.completed_tasks = [self.tasks.pop(task_index)]
                else:
                    if self.expired_tasks:
                        self.expired_tasks.append(self.tasks.pop(task_index))
                    else:
                        self.expired_tasks = [self.tasks.pop(task_index)]
            else:
                messagebox.showinfo("Error", "Task ID not found.")
            self.save_to_excel("python.xlsx")
            self.refresh_tables()
            messagebox.showinfo("Success", "Task deletion process completed!")
        else:
            messagebox.showwarning("Warning", "Please enter the Task ID to delete.")

    def update_task(self):
        id_to_update_delete = self.id_to_update_delete_entry.get()
        new_id = self.new_id_entry.get()
        title = self.title_entry.get()
        description = self.description_entry.get()
        deadline = self.deadline_entry.get()

        if id_to_update_delete:
            task_index = self.search_task_id(id_to_update_delete)
            if task_index is not None:
                if new_id and title and description and deadline:
                    self.tasks[task_index] = (new_id, title, description, deadline)
                    self.save_to_excel("python.xlsx")
                    self.refresh_tables()
                    messagebox.showinfo("Success", "Task updated successfully!")
                else:
                    messagebox.showwarning("Warning", "Please fill in all fields.")
            else:
                messagebox.showinfo("Error", "Task ID not found.")
        else:
            messagebox.showwarning("Warning", "Please enter the Task ID to update.")
    def search_task_id(self, task_id):
        for index, task in enumerate(self.tasks):
            if task[0] == task_id:
                return index
        return None
    def save_deleted_task(self, task):
        with pd.ExcelWriter("python.xlsx", mode='a') as writer:
            df = pd.DataFrame([task], columns=["ID", "Title", "Description", "Deadline"])
            df.to_excel(writer, index=False, sheet_name="Deleted task")

    def save_to_excel(self, filename):
        with pd.ExcelWriter(filename) as writer:
            df1 = pd.DataFrame(self.tasks, columns=["ID", "Title", "Description", "Deadline"])
            df1.to_excel(writer, index=False, sheet_name="Original task")
    
            df2 = pd.DataFrame(self.completed_tasks, columns=["ID", "Title", "Description", "Deadline"])
            df2.to_excel(writer, index=False, sheet_name="Completed task")
    
            df3 = pd.DataFrame(self.expired_tasks, columns=["ID", "Title", "Description", "Deadline"])
            df3.to_excel(writer, index=False, sheet_name="Expired task")

    def add_task(self):
        new_id = self.new_id_entry.get()
        title = self.title_entry.get()
        description = self.description_entry.get()
        deadline = self.deadline_entry.get()
    
        if new_id and title and description and deadline:
            self.tasks.append((new_id, title, description, deadline))
            self.save_to_excel("python.xlsx")
            self.refresh_tables()
            self.root.update() 
            self.root.update_idletasks()  # Force immediate GUI update
            messagebox.showinfo("Success", "Task added successfully!")
        else:
            messagebox.showwarning("Warning", "Please fill in all fields.")


    def load_tasks_to_table(self, sheet_name, tasks_table):
        if sheet_name == "Original task":
            tasks = self.tasks
        elif sheet_name == "Completed task":
            tasks = self.completed_tasks
        elif sheet_name == "Expired task":
            tasks = self.expired_tasks
    
        if tasks_table and tasks:  # Check if both tasks_table and tasks exist
            # Clear the existing items in the table
            for item in tasks_table.get_children():
                tasks_table.delete(item)
    
            # Populate the table with task data
            for task in tasks:
                remaining_time = self.calculate_remaining_time(task[3])
                tasks_table.insert("", "end", values=(task[0], task[1], task[2], task[3], remaining_time))
        else:
            # Handle the case where either tasks_table or tasks is None
            print("Error: Unable to load tasks to table. Table or tasks data is missing.")

    def load_tasks_from_excel(self, filename):
        try:
            wb = load_workbook(filename, read_only=True)
            tasks = []
            completed_tasks = []
            expired_tasks = []
            for sheet_name in wb.sheetnames:
                sheet = wb[sheet_name]
                for row in sheet.iter_rows(min_row=2, values_only=True):
                    if sheet_name == "Original task":
                        tasks.append(row)
                    elif sheet_name == "Completed task":
                        completed_tasks.append(row)
                    elif sheet_name == "Expired task":
                        expired_tasks.append(row)
            return tasks, completed_tasks, expired_tasks
        except FileNotFoundError:+
            return [], [], []

    def calculate_remaining_time(self, deadline_str):
        deadline = datetime.strptime(deadline_str, "%d-%m-%Y")
        remaining_time = deadline - datetime.now()
        return remaining_time.days

    def is_task_expired(self, deadline_str):
        deadline = datetime.strptime(deadline_str, "%d-%m-%Y")
        return datetime.now().date() > deadline.date()

    def refresh_tables(self):
        for sheet_name in ["Original task", "Completed task", "Expired task"]:
            for frame in self.notebook.winfo_children():
                if sheet_name in frame.winfo_name():
                    for child in frame.winfo_children():
                        if isinstance(child, ttk.Treeview):
                            self.load_tasks_to_table(sheet_name, child)

if __name__ == "__main__":
    TaskManager()
